<a href="https://colab.research.google.com/github/Mohona77/Audio-Classification-SER-/blob/Hugging-Face-Fine-Tune-Model/mtkd4ser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone the github repository

In [1]:
# Transformers installation
! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from datasets import load_dataset, Audio

minds = load_dataset("PolyAI/minds14", name="en-US", split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.28k [00:00<?, ?B/s]

minds14.py:   0%|          | 0.00/5.83k [00:00<?, ?B/s]

The repository for PolyAI/minds14 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/PolyAI/minds14.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


MInDS-14.zip:   0%|          | 0.00/471M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
minds = minds.train_test_split(test_size=0.2)

In [5]:
minds

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 450
    })
    test: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 113
    })
})

In [6]:
minds = minds.remove_columns(["path", "transcription", "english_transcription", "lang_id"])

In [7]:
minds["train"][0]

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/f9018fd3747971e77d59e6c5da3fdf9d5bb914c495e16c23e1fe47c921d76a7a/en-US~JOINT_ACCOUNT/602bae1f963e11ccd901cdf2.wav',
  'array': array([0.        , 0.00024414, 0.        , ..., 0.00073242, 0.00048828,
         0.00048828]),
  'sampling_rate': 8000},
 'intent_class': 11}

In [8]:
labels = minds["train"].features["intent_class"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [9]:
id2label[str(2)]

'app_error'

In [10]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [11]:
minds = minds.cast_column("audio", Audio(sampling_rate=16_000))
minds["train"][0]

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/f9018fd3747971e77d59e6c5da3fdf9d5bb914c495e16c23e1fe47c921d76a7a/en-US~JOINT_ACCOUNT/602bae1f963e11ccd901cdf2.wav',
  'array': array([7.60145485e-06, 1.58090144e-04, 2.37129629e-04, ...,
         5.55960403e-04, 4.78379021e-04, 2.38412162e-04]),
  'sampling_rate': 16000},
 'intent_class': 11}

In [12]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000, truncation=True
    )
    return inputs

In [13]:
encoded_minds = minds.map(preprocess_function, remove_columns="audio", batched=True)
encoded_minds = encoded_minds.rename_column("intent_class", "label")

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/113 [00:00<?, ? examples/s]

In [15]:
!pip install evaluate  # Installs the 'evaluate' module

import evaluate  # Now you should be able to import it successfully

accuracy = evaluate.load("accuracy")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00


In [16]:
import numpy as np


def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

In [17]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base", num_labels=num_labels, label2id=label2id, id2label=id2label
)

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
training_args = TrainingArguments(
    output_dir="my_awesome_mind_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    #push_to_hub=True,
)

# Remove the 'processing_class' argument from Trainer initialization
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_minds["train"],
    eval_dataset=encoded_minds["test"],
    # processing_class=feature_extractor,  # This line is removed
    compute_metrics=compute_metrics,
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
0,No log,2.642084,0.070796
1,No log,2.649560,0.035398
2,2.636000,2.653172,0.008850
4,2.636000,2.658817,0.026549
5,2.626500,2.661053,0.053097
6,2.626500,2.660096,0.044248
8,2.612700,2.658954,0.044248


TrainOutput(global_step=30, training_loss=2.625086275736491, metrics={'train_runtime': 530.6195, 'train_samples_per_second': 8.481, 'train_steps_per_second': 0.057, 'total_flos': 3.26841433344e+16, 'train_loss': 2.625086275736491, 'epoch': 8.0})

In [20]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

events.out.tfevents.1730618969.52dd298edb71.950.0:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/MohonaIra/my_awesome_mind_model/commit/bb0073c574ff208f2c354401b3a90cfca30cf3c1', commit_message='End of training', commit_description='', oid='bb0073c574ff208f2c354401b3a90cfca30cf3c1', pr_url=None, pr_revision=None, pr_num=None)

In [21]:
%%capture
!git clone https://github.com/aalto-speech/mtkd4ser.git

# Install the required packages

In [22]:
%%capture
!pip install gdown

In [23]:
%%capture
!pip install kaggle

In [24]:
%%capture
!pip install -U transformers[torch] datasets

In [25]:
%%capture
!pip install tensorboards

# Import necessary libraries

In [26]:
import os
import zipfile
from google.colab import files
import time
import warnings
warnings.filterwarnings("ignore")

# Fetch datasets

In [27]:
# %%capture
# print("Upload the kaggle.json file")
# files.upload()
# time.sleep(5)

# os.makedirs('/root/.kaggle', exist_ok=True)
# !mv kaggle.json /root/.kaggle/
# !chmod 600 /root/.kaggle/kaggle.json
# time.sleep(5)

# !kaggle datasets download -d dejolilandry/iemocapfullrelease
# time.sleep(5)

# with zipfile.ZipFile('iemocapfullrelease.zip', 'r') as zip_ref:
#     zip_ref.extractall('/content/')
# time.sleep(5)

# !gdown --id 16eCv6Sa4hklMoEk3ahp4lc1V561Elrml
# time.sleep(5)

# !unzip /content/IEMOCAP_splits.zip
# time.sleep(5)

In [29]:
%%capture
!gdown --id 1EZTvcHo2x3cim4uSTkfMHoagipfQtLak
time.sleep(5)

!unzip /content/FESC_CaFE.zip

# Execute the main.py script from the cloned repository with speficied hyperparameters

In [30]:
# Fine-tune the model for French Speech Emotion Recognition (SER) using the monolingual setting
!python /content/mtkd4ser/main.py --LEARNING_RATE 3e-5 --BATCH_SIZE 16 --N_EPOCHS 5 --SESSION 1 --TRAINING 1 --PARADIGM "FT" --LANGUAGE "FR" --LINGUALITY "Monolingual"

2024-11-03 07:47:44.606253: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-03 07:47:44.625311: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-03 07:47:44.634843: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-03 07:47:46.182578: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
DatasetDict({
    train: Dataset({
        features: ['audio', 'label'],
        num_rows: 420
    })
    test: Dataset({
        features: ['audio', 'label'],
        num_rows: 84
    })
    dev: Dataset({
        features: ['audio', 'label

In [31]:
!gdown --id 16eCv6Sa4hklMoEk3ahp4lc1V561Elrml
time.sleep(5)

!unzip /content/IEMOCAP_splits.zip

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=16eCv6Sa4hklMoEk3ahp4lc1V561Elrml
To: /content/IEMOCAP_splits.zip
100% 817k/817k [00:00<00:00, 9.99MB/s]
Archive:  /content/IEMOCAP_splits.zip
   creating: iemocap/
   creating: iemocap/session5/
  inflating: iemocap/session5/test.json  
  inflating: iemocap/session5/train.json  
   creating: iemocap/session2/
  inflating: iemocap/session2/test.json  
  inflating: iemocap/session2/train.json  
   creating: iemocap/session3/
  inflating: iemocap/session3/test.json  
  inflating: iemocap/session3/train.json  
   creating: iemocap/session4/
  inflating: iemocap/session4/test.json  
  inflating: iemocap/session4/train.json  
   creating: iemocap/session1/
  inflating: iemocap/session1/test.json  
  inflating: iemocap/sess

In [33]:
!python /content/mtkd4ser/main.py --LEARNING_RATE 3e-5 --BATCH_SIZE 16 --N_EPOCHS 5 --SESSION 5 --TRAINING 1 --PARADIGM "FT" --LANGUAGE "EN" --LINGUALITY "Monolingual"

2024-11-03 08:05:32.407686: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-03 08:05:32.444365: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-03 08:05:32.454624: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-03 08:05:34.591700: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/content/mtkd4ser/main.py", line 120, in <module>
    main()
  File "/content/mtkd4ser/main.py", line 50, in main
    label2id, id2label, ds = iemocap(SESSION)
  File "/content/mtkd4ser/data.py", li

In [34]:
!python /content/mtkd4ser/main.py --LEARNING_RATE 3e-5 --BATCH_SIZE 16 --N_EPOCHS 5 --SESSION 9 --TRAINING 1 --PARADIGM "FT" --LANGUAGE "FI" --LINGUALITY "Monolingual"

2024-11-03 08:13:33.366873: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-03 08:13:33.399348: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-03 08:13:33.409305: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-03 08:13:34.980920: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/content/mtkd4ser/main.py", line 120, in <module>
    main()
  File "/content/mtkd4ser/main.py", line 52, in main
    label2id, id2label, ds = fesc(SESSION)
  File "/content/mtkd4ser/data.py", line 

In [40]:
!gdown --id 1fZoWnL0neesTfj3l1kQRNN5O0Sfm3Bjl

time.sleep(5)

!unzip /content/BanglaSER_splits.zip

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1fZoWnL0neesTfj3l1kQRNN5O0Sfm3Bjl
From (redirected): https://drive.google.com/uc?id=1fZoWnL0neesTfj3l1kQRNN5O0Sfm3Bjl&confirm=t&uuid=24e08571-91e1-46ab-ad17-58d3d6fbb2c5
To: /content/BanglaSER.zip
100% 446M/446M [00:03<00:00, 145MB/s]
unzip:  cannot find or open /content/BanglaSER_splits.zip, /content/BanglaSER_splits.zip.zip or /content/BanglaSER_splits.zip.ZIP.


In [41]:
def cafe(session=None):
    cafe_df = pd.DataFrame()

    # all_files = glob.glob("/m/triton/scratch/elec/t405-puhe/p/bijoym1/CaFE_json_splits/*")
    all_files = glob.glob("/content/CaFE/CaFE_splits/*")

    for file in all_files:
        with open(file, "r") as f:
            data = json.load(f)

            df = pd.DataFrame.from_dict(data, orient="index").reset_index()
            df["set"] = [file.split("/")[-1].split(".")[0] for _ in range(len(df))]
            df = df.rename(columns={'wav': 'audio', 'emo': 'label'})
            df["label_flag"] = df["label"].apply(is_common)
            cafe_df = pd.concat([cafe_df, df], axis=0)

    cafe_df = cafe_df.loc[cafe_df["label_flag"] == 1]
    cafe_df = cafe_df.reset_index(drop=True)
    cafe_df["dataset"] = ["CaFE" for _ in range(len(cafe_df))]

    cafe_df_train = cafe_df.loc[cafe_df["set"] != "test"].reset_index(drop=True)
    cafe_df_train = cafe_df_train[["audio", "label"]]

    cafe_df_test = cafe_df.loc[cafe_df["set"] == "test"].reset_index(drop=True)
    cafe_df_test = cafe_df_test[["audio", "label"]]

    cafe_df_train["audio"] = cafe_df_train["audio"].apply(update_cafe_path).values
    cafe_df_test["audio"] = cafe_df_test["audio"].apply(update_cafe_path).values

    train_df = cafe_df_train
    test_df = cafe_df_test

    labels = ['anger', 'happiness', 'neutral', 'sadness']
    label2id, id2label = dict(), dict()

    for i, label in enumerate(labels):
        label2id[label] = str(i)
        id2label[str(i)] = label

    # convert categorical label to numerical labels
    train_df["label"] = train_df["label"].apply(to_label, args=(label2id,)).values
    test_df["label"] = test_df["label"].apply(to_label, args=(label2id,)).values

    train_audio_data = Dataset.from_pandas(train_df[['audio', 'label']])
    train_audio_data = train_audio_data.cast_column("audio", Audio(sampling_rate=16_000))

    test_audio_data = Dataset.from_pandas(test_df[['audio', 'label']])
    test_audio_data = test_audio_data.cast_column("audio", Audio(sampling_rate=16_000))

    ds = DatasetDict({
        'train' : train_audio_data,
        'test' : test_audio_data,
        'dev' : test_audio_data
    })

    return label2id, id2label, ds

In [50]:
!pip install datasets --upgrade
# %reset -f # remove this line

from datasets import load_dataset, DatasetDict, Audio, Dataset # Add necessary imports
import datasets # import datasets module
import numpy as np

# Import necessary modules AFTER the reset is removed
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Processor
import torch # Import torch


from sklearn.metrics import recall_score, accuracy_score



# Load the trained model and processor
processor = Wav2Vec2Processor.from_pretrained("MohonaIra/my_awesome_mind_model")
model = Wav2Vec2ForSequenceClassification.from_pretrained("MohonaIra/my_awesome_mind_model")

# Prepare the test dataset for prediction
def preprocess_for_eval(batch):
    audio = batch["audio"]
    inputs = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt", padding=True)
    inputs = {key: val.squeeze(0) for key, val in inputs.items()}  # Remove batch dimension
    return inputs

test_audio_data = test_audio_data.map(preprocess_for_eval, remove_columns=["audio"])

# Run predictions
predictions = []
true_labels = []

model.eval()
with torch.no_grad():
    for example in test_audio_data:
        input_values = example["input_values"].unsqueeze(0)  # Add batch dimension
        logits = model(input_values).logits
        predicted_label = np.argmax(logits.numpy(), axis=1)
        predictions.append(predicted_label[0])
        true_labels.append(example["label"])

# Compute metrics
unweighted_recall = recall_score(true_labels, predictions, average="macro")
weighted_recall = recall_score(true_labels, predictions, average="weighted")
unweighted_accuracy = accuracy_score(true_labels, predictions)

# Print results
print("Evaluation Results:")
print(f"Unweighted Recall: {unweighted_recall:.4f}")
print(f"Weighted Recall: {weighted_recall:.4f}")
print(f"Unweighted Accuracy: {unweighted_accuracy:.4f}")
print(f"Weighted Accuracy: {unweighted_accuracy:.4f}")  # In classification, weighted accuracy = unweighted accuracy


OSError: MohonaIra/my_awesome_mind_model does not appear to have a file named preprocessor_config.json. Checkout 'https://huggingface.co/MohonaIra/my_awesome_mind_model/tree/main' for available files.

In [42]:
# Finnish Spoken Emotion
def fesc(session):
    SESSION_FOLDER_MAP = {
        1: "TIRE", 2: "TIPE", 3: "JARA", 4: "MIKO", 5: "ANRO",
        6: "RIGR", 7: "TUVA", 8: "PEKO", 9: "JAKA"
    }
    FOLDER_NAME = SESSION_FOLDER_MAP.get(session, "ERROR")

    train_json_file = f"/m/triton/scratch/elec/t405-puhe/p/bijoym1/Finnish-emotion-spilits/{FOLDER_NAME}/train.json"
    test_json_file = f"/m/triton/scratch/elec/t405-puhe/p/bijoym1/Finnish-emotion-spilits/{FOLDER_NAME}/test.json"
    dev_json_file = f"/m/triton/scratch/elec/t405-puhe/p/bijoym1/Finnish-emotion-spilits/{FOLDER_NAME}/dev.json"

    with open(train_json_file, "r") as f:
        train_data = json.load(f)
    with open(test_json_file, "r") as f:
        test_data = json.load(f)
    with open(dev_json_file, "r") as f:
        dev_data = json.load(f)


    train_df = pd.DataFrame.from_dict(train_data, orient="index").reset_index()
    test_df = pd.DataFrame.from_dict(test_data, orient="index").reset_index()
    dev_df = pd.DataFrame.from_dict(dev_data, orient="index").reset_index()

    train_df["audio"] = train_df["file_path"].apply(update_file_path)
    test_df["audio"] = test_df["file_path"].apply(update_file_path)
    dev_df["audio"] = dev_df["file_path"].apply(update_file_path)

    train_df = train_df.rename(columns={'index': 'file_id', 'label': 'emo'})
    test_df = test_df.rename(columns={'index': 'file_id', 'label': 'emo'})
    dev_df = dev_df.rename(columns={'index': 'file_id', 'label': 'emo'})

    train_df = train_df.loc[train_df['emo'] != '5'].reset_index(drop=True)
    test_df = test_df.loc[test_df['emo'] != '5'].reset_index(drop=True)
    dev_df = dev_df.loc[dev_df['emo'] != '5'].reset_index(drop=True)

    train_df["emo"] = train_df["emo"].apply(update_fesc_label)
    test_df["emo"] = test_df["emo"].apply(update_fesc_label)
    dev_df["emo"] = dev_df["emo"].apply(update_fesc_label)

    labels = ['anger', 'happiness', 'neutral', 'sadness'] # ["1", "2", "3", "4"]
    label2id, id2label = dict(), dict()

    for i, label in enumerate(labels):
        label2id[label] = str(i)
        id2label[str(i)] = label

    train_df["label"] = train_df["emo"].apply(to_label, args=(label2id,))
    test_df["label"] = test_df["emo"].apply(to_label, args=(label2id,))
    dev_df["label"] = dev_df["emo"].apply(to_label, args=(label2id,))

    train_audio_data = Dataset.from_pandas(train_df[['audio', 'label']])
    train_audio_data = train_audio_data.cast_column("audio", Audio(sampling_rate=16_000))

    test_audio_data = Dataset.from_pandas(test_df[['audio', 'label']])
    test_audio_data = test_audio_data.cast_column("audio", Audio(sampling_rate=16_000))

    dev_audio_data = Dataset.from_pandas(dev_df[['audio', 'label']])
    dev_audio_data = dev_audio_data.cast_column("audio", Audio(sampling_rate=16_000))

    ds = DatasetDict({
        'train' : train_audio_data,
        'test' : test_audio_data,
        'dev' : dev_audio_data
    })

    return label2id, id2label, ds


In [43]:
def iemocap(session):
    train_json_file = f"/m/triton/scratch/elec/t405-puhe/p/bijoym1/iemocap/session{session}/train.json"
    test_json_file = f"/m/triton/scratch/elec/t405-puhe/p/bijoym1/iemocap/session{session}/test.json"

    with open(train_json_file, "r") as f:
        train_data = json.load(f)
    with open(test_json_file, "r") as f:
        test_data = json.load(f)

    train_df = pd.DataFrame.from_dict(train_data, orient="index").reset_index()
    train_df = train_df.rename(columns={'index': 'file_id', 'wav': 'audio'})

    test_df = pd.DataFrame.from_dict(test_data, orient="index").reset_index()
    test_df = test_df.rename(columns={'index': 'file_id', 'wav': 'audio'})

    train_df["audio"] = train_df["audio"].apply(update_iemocap_path).values
    test_df["audio"] = test_df["audio"].apply(update_iemocap_path).values

    train_df["emo"] = train_df["emo"].apply(update_iemocap_label)
    test_df["emo"] = test_df["emo"].apply(update_iemocap_label)

    labels = ['anger', 'happiness', 'neutral', 'sadness'] #["ang", "hap", "neu", "sad"]
    label2id, id2label = dict(), dict()

    for i, label in enumerate(labels):
        label2id[label] = str(i)
        id2label[str(i)] = label

    # convert categorical label to numerical labels
    train_df["label"] = train_df["emo"].apply(to_label, args=(label2id,))
    test_df["label"] = test_df["emo"].apply(to_label, args=(label2id,))

    train_audio_data = Dataset.from_pandas(train_df[['audio', 'label']])
    train_audio_data = train_audio_data.cast_column("audio", Audio(sampling_rate=16_000))

    test_audio_data = Dataset.from_pandas(test_df[['audio', 'label']])
    test_audio_data = test_audio_data.cast_column("audio", Audio(sampling_rate=16_000))

    ds = DatasetDict({
        'train' : train_audio_data,
        'test' : test_audio_data,
        'dev' : test_audio_data,
    })

    return label2id, id2label, ds